In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [2]:
!pip install -q tensorflow-datasets

In [3]:
import tensorflow_datasets as tfds
imdb,info=tfds.load("imdb_reviews",with_info=True,as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteI9D7EH/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteI9D7EH/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteI9D7EH/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
import numpy as np
train_data,test_data=imdb['train'],imdb['test']

In [5]:
training_sentences=[]
testing_sentences=[]
training_labels=[]
testing_labels=[]

for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())


for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())


In [6]:
training_labels_final=np.array(training_labels)
testing_labels=np.array(testing_labels)

In [7]:
vocab_size=10000
embedding_dim=16
max_legth=120
trunc_type='post'
oov_tok="<OOV>"

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(training_sentences)
padded=pad_sequences(sequences,maxlen=max_legth,truncating=trunc_type)

In [10]:
testing_sequences=tokenizer.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(testing_sequences,maxlen=max_legth)

In [11]:
testing_padded[1]

array([  26, 5473,    3,    9,    4, 3319,  339,   48,    6, 2285,  199,
          2,  113,   27,   48,   78,  981,   42,    2, 4096,  113,   13,
          7,  881,    5,    4, 2324,   25,   37,  927,   87,   34, 4450,
         26, 1539, 5112,    3,   75,   25,  117,  180,  250,  734,   10,
       1309, 7077,   42,   24,    8,    8,   12,    7,    4, 1758,   20,
          3,   11,   64, 4269,  254,  935,    9,  363,  436,    6, 2767,
         10,   46,   10,    7,   30,    5,    1,   92,  736,  109,    3,
          1,  112,    5,   26,    1, 3614, 1777, 1185,  119, 1760,    1,
        529,    9,   63,  654,    1,    7,  339,   67,    6, 1083,    1,
          1,    9, 1324,    5,   26, 2698,  101,    3,    7,  444,  759,
          2,  356,    5,    2, 1037,    5,    1,  971,    1,    1],
      dtype=int32)

In [14]:
#Defining Our Neural Network ----->
model=tf.keras.Sequential([
                           tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_legth),
                           tf.keras.layers.GlobalAveragePooling1D(),
                           tf.keras.layers.Dense(6,activation='relu'),
                           tf.keras.layers.Dense(1,activation='sigmoid')

])

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [19]:
num_epochs=10
model.fit(padded,training_labels_final,epochs=num_epochs,validation_data=(testing_padded,testing_labels))

Epoch 1/10
782/782 [==============================] - 7s 7ms/step - loss: 0.6537 - accuracy: 0.5957 - val_loss: 0.4080 - val_accuracy: 0.8394
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3637 - accuracy: 0.8570 - val_loss: 0.3373 - val_accuracy: 0.8546
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2850 - accuracy: 0.8845 - val_loss: 0.3283 - val_accuracy: 0.8581
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2465 - accuracy: 0.9044 - val_loss: 0.3385 - val_accuracy: 0.8551
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2144 - accuracy: 0.9185 - val_loss: 0.3516 - val_accuracy: 0.8509
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1923 - accuracy: 0.9297 - val_loss: 0.3721 - val_accuracy: 0.8471
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1732 - accuracy: 0.9405 - val_loss: 0.3961 - val_accuracy: 0.8416
Epoch 

In [20]:
e=model.layers[0]
weights=e.get_weights()[0]
print(weights.shape) #SHape-->(vocab_size,embedding_dim)

(10000, 16)


In [21]:
reverse_word_index=dict([(value,key) for (key,value) in word_index.items()])

In [22]:
reverse_word_index

{1: '<OOV>',
 2: 'the',
 3: 'and',
 4: 'a',
 5: 'of',
 6: 'to',
 7: 'is',
 8: 'br',
 9: 'in',
 10: 'it',
 11: 'i',
 12: 'this',
 13: 'that',
 14: 'was',
 15: 'as',
 16: 'for',
 17: 'with',
 18: 'movie',
 19: 'but',
 20: 'film',
 21: "'s",
 22: 'on',
 23: 'you',
 24: 'not',
 25: 'are',
 26: 'his',
 27: 'he',
 28: 'have',
 29: 'be',
 30: 'one',
 31: 'all',
 32: 'at',
 33: 'by',
 34: 'they',
 35: 'an',
 36: 'who',
 37: 'so',
 38: 'from',
 39: 'like',
 40: 'her',
 41: "'t",
 42: 'or',
 43: 'just',
 44: 'there',
 45: 'about',
 46: 'out',
 47: "'",
 48: 'has',
 49: 'if',
 50: 'some',
 51: 'what',
 52: 'good',
 53: 'more',
 54: 'very',
 55: 'when',
 56: 'she',
 57: 'up',
 58: 'can',
 59: 'b',
 60: 'time',
 61: 'no',
 62: 'even',
 63: 'my',
 64: 'would',
 65: 'which',
 66: 'story',
 67: 'only',
 68: 'really',
 69: 'see',
 70: 'their',
 71: 'had',
 72: 'were',
 73: 'me',
 74: 'well',
 75: 'we',
 76: 'than',
 77: 'much',
 78: 'been',
 79: 'get',
 80: 'bad',
 81: 'will',
 82: 'people',
 83: 'do',

In [23]:
import io

out_v=io.open('vecs.tsv','w',encoding='utf-8')
out_m=io.open('meta.tsv','w',encoding='utf-8')
for word_num in range(1,vocab_size):
  word=reverse_word_index[word_num]
  embeddings=weights[word_num]
  out_m.write(word+'\n')
  out_v.write('\t'.join([str(x) for x in embeddings])+'\n')

out_v.close()
out_m.close()  